In [1]:
import numpy as np 
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LSTM

import string
from preprocessor import load_sonnets

sonnets = load_sonnets()

Using TensorFlow backend.


In [16]:
chars = list(string.ascii_lowercase)
#chars = list(string.ascii_letters)
chars += [",", ".", "?", "!", ":", ";", "'", "(", ")", " ", "\n", "-"]

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

train_strings = []
char_length = 41
char_skip = 3 #20

for _, s in enumerate(sonnets):
    #sonnet = "\n".join(sonnets[0]).lower()
    sonnet = "\n".join(s).lower()
    
    for i in range(0, len(sonnet) - char_length, char_skip):
        train_strings.append(sonnet[i: i+char_length])
    #break
    
x = np.zeros((len(train_strings), char_length - 1, len(chars)), dtype=np.bool)
y = np.zeros((len(train_strings), len(chars)), dtype=np.bool)

for i, train_string in enumerate(train_strings):
    for t, char in enumerate(train_string[:-1]):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[train_string[-1]]] = 1

In [44]:
#timesteps = 8
model = Sequential()
#model.add(Dense(200, input_shape=(200,)))
model.add(LSTM(200, input_shape=(char_length - 1, len(chars))))
#model.add(Dense(200))
#model.add(Activation('relu'))
#model.add(Dropout(dropout))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [45]:
#model.summary()
model.fit(x, y, batch_size=64, epochs=20)

Epoch 1/20
29325/29325 [==============================] - 74s 3ms/step - loss: 2.7800 - acc: 0.2270
Epoch 2/20
29325/29325 [==============================] - 69s 2ms/step - loss: 2.2972 - acc: 0.3328
Epoch 3/20
29325/29325 [==============================] - 74s 3ms/step - loss: 2.1422 - acc: 0.3719
Epoch 4/20
29325/29325 [==============================] - 69s 2ms/step - loss: 2.0395 - acc: 0.4000
Epoch 5/20
29325/29325 [==============================] - 68s 2ms/step - loss: 1.9516 - acc: 0.4224
Epoch 6/20
29325/29325 [==============================] - 72s 2ms/step - loss: 1.8732 - acc: 0.4409
Epoch 7/20
29325/29325 [==============================] - 73s 3ms/step - loss: 1.8132 - acc: 0.4587
Epoch 8/20
29325/29325 [==============================] - 72s 2ms/step - loss: 1.7525 - acc: 0.4712
Epoch 9/20
29325/29325 [==============================] - 73s 2ms/step - loss: 1.6956 - acc: 0.4866
Epoch 10/20
29325/29325 [==============================] - 76s 3ms/step - loss: 1.6457 - acc: 0.5005

In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [52]:
# Function equivalent to below
def generate_poem(seed = "shall i compare thee to a summer's day?\n", lines=14):
    generated = "" + seed

    # Generates 14 lines of poem
    
    for i in range(lines):
        while True:
            x_pred = np.zeros((1, char_length - 1, len(chars)))
            for t, char in enumerate(seed):
                x_pred[0, t, char_indices[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
        
            next_index = sample(preds, temperature=0.5)
            next_char = indices_char[next_index]
            seed = seed[1:] + next_char
        
            generated += next_char
            if next_char == "\n":
                #print(generated)
                #generated = ""
                break
            #generated += next_char

    return generated
    
print(generate_poem())

'''

shall i compare thee to a summer's day?
the praise the praise the reauted from thee,
but thou dest that songsed in thy reast changer
even to be more with thou mestresss dast sack of bord's new,
and stall dest in my dove them farren,
when i sey a selfer tean heart thou sood,
on the thou art and for my seathons spead,
but mine ow spest of your sild praise,
shang on my live and praises if that i not,
  but thou less it lives beauty's gaine,
when is she whor my life it me do bles,
though thy that is the tome, the recoor fir my prais,
but there my solf love so my love the wrent,
excessed thy frast with thy beture then leave,
that more preaute of the restorn love and grave,
'''

shall i compare thee to a summer's day?
that i have some thou desire thee mend,
that of this gidce then thou lov's stays,
  and i non this my out in pressed me,
  in that the pleasure of this with the elles stall despain:
the rast cons new vespech and beauty of thee,
  then thou love's see,
  but thou dest the sweet on the sun my sweat,
not beck do plazing spest of this slive,
the right fair that doth ckmen machire thee,
me hourd of my self, nor thee i hath to geet,
  thou apperition of thy my sughts the premont,
which hate the words of that stould to be,
  but i love's  and thee most can my self,
though not the rectire of thy sweet strong,



In [50]:
#seed = "shall i compare thee to a summer's day?\nthou art more lovely"
seed = "shall i compare thee to a summer's day?\n"
generated = "" + seed

# Generates 14 lines of poem (More efficient)

x_pred = np.zeros((1, char_length - 1, len(chars)))
for t, char in enumerate(seed):
    x_pred[0, t, char_indices[char]] = 1.
preds = model.predict(x_pred, verbose=0)[0]

for i in range(14):
    while True:
        next_index = sample(preds, temperature=0.2)
        next_char = indices_char[next_index]
        
        x_pred[0, :-1, :] = x_pred[:, 1:, :]
        x_pred[0, -1, :] = np.zeros(len(chars))
        x_pred[0, -1, char_indices[next_char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        
        generated += next_char
        if next_char == "\n":
            #print(generated)
            #generated = ""
            break
        #generated += next_char
        

print(generated)

shall i compare thee to a summer's day?
the ore wo das words on thy sweet love still,
though thy love ast speaking shall the rest,
why i me do thee mayst chought of thy love,
and stay the report of mort fair thou presse,
the wirnt thou art that is the pless of tind,
  and the thy banter that the sur oft so,
and that i have so my love thee my lie:
the sun i contant of this swill stand
the show thy sweet streace the sweet strings,
  thou and the strong to fair thou art and speak,
the summer end hath that she should to be,
the wiented that the store thou art and prave,
that in the surmer in my self in liess be ong that i am stall,
bet by a det an my blath that in the sweet,



In [23]:
print(y.shape)
#print(generated)

(29325, 38)

